In [1]:
import pandas as pd
import joblib

# 1. Load both saved pipelines
approval_model = joblib.load("Approval_Model.pkl")
coverage_model = joblib.load("Coverage_Model.pkl")
anomaly_model  = joblib.load("Anomaly_Model.pkl")

In [2]:
# 2. Define user-specified claim(s)
user_claims = pd.DataFrame([{
    "age": 34,
    "months_as_customer": 48,
    "vehicle_age_years": 4,
    "vehicle_make": "Perodua Myvi",
    "policy_expired_flag": 0,
    "deductible_amount": 500.0,
    "market_value": 28000.0,
    "damage_severity_score": 0.35,
    "repair_amount": 9000.0,
    "at_fault_flag": 0,
    "time_to_report_days": 3,
    "claim_reported_to_police_flag": 1,
    "license_type_missing_flag": 0,
    "num_third_parties": 1,
    "num_witnesses": 2
}])

In [3]:
 import numpy as np

# 1. Predict approval
approval_pred = approval_model.predict(user_claims)              # array([0] or [1])
approval_prob = approval_model.predict_proba(user_claims)[:, 1]  # P(approve)

print(f"Approval prediction: {approval_pred[0]} (prob={approval_prob[0]:.3f})")

Approval prediction: 1 (prob=1.000)


In [4]:
# 2. If approved (==1), predict coverage; otherwise skip
if approval_pred[0] == 1:
    coverage_pred = coverage_model.predict(user_claims)         # continuous coverage_amount
    print(f"Predicted coverage amount: {coverage_pred[0]:,.2f}")
else:
    coverage_pred = None
    print("Claim not approved → no coverage prediction")

Predicted coverage amount: 12,955.98


In [5]:
# 3. Always check for anomaly on the claim features
#    IsolationForest.predict → -1 = anomaly, 1 = normal
anomaly_scores = anomaly_model.decision_function(user_claims)   # higher → more “normal”
anomaly_flag = anomaly_model.predict(user_claims)               # -1 or 1
is_anomaly = np.where(anomaly_flag == -1, 1, 0)                 # convert to 0/1

print(f"Anomaly score: {anomaly_scores[0]:.3f}")
print(f"Anomaly flag: {is_anomaly[0]}  (1 = anomaly)")

Anomaly score: 0.192
Anomaly flag: 0  (1 = anomaly)
